<a href="https://colab.research.google.com/github/Gauravkumarlokhande/Gauravkumarlokhande/blob/main/luhn_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re  #regular expression library
import nltk # natural language toolkit
import string
import heapq #to order the sentences

in order to work with words and sentence tokenizer, download punkt

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

To work with stop words download stopwords 

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
original = """Artificial intelligence is human like intelligence machines.
              It is the study of intelligent artificial agents.
              Science and engineering to produce intelligent machines.
              Solve problems and have intelligence.
              Related to intelligent behavior.
              Developing of reasoning machines.
              Learn from mistakes and successes.
              Artificial intelligence is related to reasing in everyday situations."""
              


In [20]:
original

'Artificial intelligence is human like intelligence machines.\n              It is the study of intelligent artificial agents.\n              Science and engineering to produce intelligent machines.\n              Solve problems and have intelligence.\n              Related to intelligent behavior.\n              Developing of reasoning machines.\n              Learn from mistakes and successes.\n              Artificial intelligence is related to reasing in everyday situations.'

Now we have many extra spaces here. so to remove the extra spaces we write the following code and just replace the big spaces by small spaces.

In [21]:
original=re.sub(r'\s+'," ",original) # the + tells that there may another spaces in the passage which allows the command to go through whole passage

In [22]:
original

'Artificial intelligence is human like intelligence machines. It is the study of intelligent artificial agents. Science and engineering to produce intelligent machines. Solve problems and have intelligence. Related to intelligent behavior. Developing of reasoning machines. Learn from mistakes and successes. Artificial intelligence is related to reasing in everyday situations.'

now we need to check for the stopwords in english language so we use the following command.

In [23]:
stop=nltk.corpus.stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Now we define a function to tokenize the words and convert to lower case and remosve stopwords

In [24]:
def process(text):
  new_text=text.lower()  # covert all the words in to lower case
  tokens=[]  #create a empty list by name tokens
  for token in nltk.word_tokenize(new_text):
    tokens.append(token)     # append each tokenized word in the empty list
  tokens=[word for word in tokens if word not in stop and word not in string.punctuation]   #we remove the stopwords and punctuation mark
  new_text=" ".join(element for element in tokens) # join all the words in to a from of sentence which are in a form of list
  return new_text


In [25]:
new_text=process(original)
new_text

'artificial intelligence human like intelligence machines study intelligent artificial agents science engineering produce intelligent machines solve problems intelligence related intelligent behavior developing reasoning machines learn mistakes successes artificial intelligence related reasing everyday situations'

**Calculating the sentence score** 

In [53]:
def calculate_sent_score(sentences,important_words,distance): # the second parameter is to return the top 5 important words
  scores=[]  # empty list
  sentence_index=0
  for sentence in [nltk.word_tokenize(sentence) for sentence in sentences]: # gives list of words in every sentence
    #print('-------------')
    #print(sentence)  # gives each sentence in a list form
    word_index=[]  # empty list to ente rthe index
    for word in important_words:
      #print(word) 
      try:
        word_index.append(sentence.index(word)) # add the index of the word which is present in the list
      #now when the word is not there in the list it will return error
      except ValueError:
        pass           # this will avoid the value for the word that are not present in the list and go for another word
    word_index.sort()
    #print(word_index)
    if len(word_index)==0:  #if the difference between the indexes is 0 then we skip that sentence
      continue
    group_list=[]
    group=[word_index[0]]   #to return word index at position 0
    # [1,2,3]
    i=1
    while i<len(word_index):    # to go through each word index in each sentence   # lenght is 3
      if word_index[i]-word_index[i-1]<distance: # here i=1 and i-1=0  # if the difference is less than distance then we execute the loop
        group.append(word_index[i])
      else:
        group_list.append(group[:])   # or else the distance is not less than the difference then we store the index in group_list
        group=[word_index[i]]
        #print('group',group)
      i+=1                          # when the value of index i is not less then the length of word index 
      group_list.append(group)
      #print('all_groups',group_list)
      max_group_score=0
      for g in group_list:
        #print(g)
        important_words_in_group=len(g)
        total_words_in_group=g[-1]-g[0]+1
        score=1.0*important_words_in_group**2/total_words_in_group
        #print('group_score',score)

        if score>max_group_score:
          max_group_score=score

      scores.append((max_group_score,sentence_index))
      sentence_index+=1
    
  #print('final score',scores)
  return scores

  

     
         



***Summarizing the text***

In [57]:
def summarize(text, top_n_words,distance,number_of_sentences):
  original_sent=[sentence for sentence in nltk.sent_tokenize(text)]                         #tokenize the whole sentence and store in a list
  formatted_sentence=[process(original_sentence) for original_sentence in original_sent ]    # to apply the whole procedure above on just sentences.
  words=[word for sentence in formatted_sentence for word in nltk.word_tokenize(sentence)]   # tokenize the sentences in the form of words
  frequency=nltk.FreqDist(words)                                                             # calculate the frequency for each word (how many times repeated. more the frequency, more is the word imp)
  top_n_words=[word[0] for word in frequency.most_common(top_n_words)]                       # to return top good frequency words
  sentences_score=calculate_sent_score(formatted_sentence,top_n_words,distance)
  #print(sentences_score)
  best_sentences=heapq.nlargest(number_of_sentences,sentences_score)
  #print(best_sentences)
  best_sentences=[original_sent[i] for (score,i) in best_sentences]
  #print(best_sentences)
  return best_sentences, original_sent, sentences_score


In [58]:
best_sentences, original_sent, sentences_score=summarize(original,5,2,3)

here in final score: (1.0,2) means sentence 2 has score 1.0

In [59]:
best_sentences

['Learn from mistakes and successes.',
 'Developing of reasoning machines.',
 'Related to intelligent behavior.']

In [60]:
sentences_score

[(2.0, 0), (2.0, 1), (2.0, 2), (2.0, 3), (2.0, 4), (2.0, 5), (3.0, 6)]

***implementation of the code***

In [61]:
from IPython.core.display import HTML

In [62]:
text=''
display(HTML(f'<h2>summary</h2>'))
for sentence in original_sent:
  if sentence in best_sentences:
    text+=''+sentence.replace(sentence,f"<mark>{sentence}</mark>")
  else:
    text+=''+sentence
display(HTML(f"""{text}"""))